<h1> Segmenting and Clustering Neighbourhoods in Toronto - Week 3 - Part 3 </h1>

<h2>Import required libraries</h2>

In [19]:
import pandas as pd
import numpy as np
import requests as req
import csv
from bs4 import BeautifulSoup as bsp

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes

import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

<h2>Get the data from the Wikipedia page for the list of postal codes in Canada</h2>

In [2]:
scrape = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(scrape)

<h3>Read the data on the page</h3>

In [3]:
df[0]

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


<h2>Convert the above data into a dataframe and clean it.</h2>

<h3>Convert the dataframe</h3>

In [4]:
df_tronto=pd.DataFrame(df[0])
df_tronto.rename(index=int, columns={0:'PostalCode', 1:'Borough', 2:'Neighborhood'}, inplace=True)

df_tronto.head()

,PostalCode,Borough,Neighborhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


<h3>Clean the dataframe</h3>

In [5]:
df_tc=df_tronto[df_tronto['Borough']!='Not assigned'].copy()
df_tc['Neighborhood'].replace('Not assigned',  df_tronto.Borough, inplace=True)
df_tc.drop(0, inplace=True)

df_tc.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


<h2>Grouping the dataframe to combine the postal codes</h2>

In [6]:
df_tcf= df_tc.groupby('PostalCode').agg({'Borough':'first', 'Neighborhood': ', '.join,}).reset_index()

df_tcf.head()

,PostalCode,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


<h1>Get the CSV file containing the Geospatial data</h1>

In [7]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2019-05-26 17:58:34--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.72.228|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2019-05-26 17:58:34--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-05-26 17:58:35--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-05-26 17:58:41--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y

<h2>Create the data frame using the data</h2>

In [8]:
df_gc = pd.read_csv('GeoCord.csv')
df_gc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h2>Change Postal Code name to PostalCode in order to merge the 2 files</h2>

In [9]:
df_gc.rename(index=int, columns={'Postal Code':'PostalCode'}, inplace=True)

df_gc.head(2)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


<h2>Merge the two csv files</h2>

In [34]:
df_mer=pd.merge(df_tcf, df_gc, how='left', on='PostalCode')

df_mer.head(12)

,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711
3,M1G,Woburn,Scarborough,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476
5,M1J,Scarborough Village,Scarborough,43.744734,-79.239476
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough,43.727929,-79.262029
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough,43.711112,-79.284577
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough,43.716316,-79.239476
9,M1N,"Birch Cliff, Cliffside West",Scarborough,43.692657,-79.264848


<h1>Create a map of Toronto</h1>

In [111]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_mer['Latitude'], df_mer['Longitude'], df_mer['Borough'], df_mer['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3131',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

<h2>Create a new dataframe for selected Borough (North York)</h2>

In [112]:
northyork_data = df_mer[df_mer['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head(10)

,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M2H,Hillcrest Village,North York,43.803762,-79.363452
1,M2J,"Fairview, Henry Farm, Oriole",North York,43.778517,-79.346556
2,M2K,Bayview Village,North York,43.786947,-79.385975
3,M2L,"Silver Hills, York Mills",North York,43.757490,-79.374714
4,M2M,"Newtonbrook, Willowdale",North York,43.789053,-79.408493
5,M2N,Willowdale South,North York,43.770120,-79.408493
6,M2P,York Mills West,North York,43.752758,-79.400049
7,M2R,Willowdale West,North York,43.782736,-79.442259
8,M3A,Parkwoods,North York,43.753259,-79.329656
9,M3B,Don Mills North,North York,43.745906,-79.352188


<h2>Add Foursquare credientials and version (hidden cell)</h2>

In [113]:
# The code was removed by Watson Studio for sharing.

<h2>Get coordinates for North York</h2>

In [114]:
address_ny = 'North York,Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address_ny)
latitude_ny = location.latitude
longitude_ny = location.longitude

print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_ny, longitude_ny))

The geograpical coordinate of Scarborough are 43.7708175, -79.4132998.


<h2>Create a map and add markers</h2>

In [115]:
map_ny = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=12.5)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3131',
        fill_opacity=0.7).add_to(map_ny)  
    
map_ny

<h1>Get the top Venues in the Hillcrest Village Neighborhood limited to 100</h1>

In [116]:
# Get the Coordinates for the Hillcrest Village Neighborhood
hv_latitude = northyork_data.loc[0, 'Latitude'] # neighbourhood latitude value
hv_longitude = northyork_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               hv_latitude, 
                                                               hv_longitude))

Latitude and longitude values of "Hillcrest Village" are 43.8037622, -79.3634517.


<h2>Make the Foursquared call and get the JSON file</h2>

In [117]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ny, longitude_ny, VERSION, radius, LIMIT)

hv_json = requests.get(url).json()
hv_json

{'meta': {'code': 200, 'requestId': '5ceafa479fb6b775800fac33'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-563d44fccd1044ad67a744fb-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ce941735',
         'name': 'Seafood Restaurant',
         'pluralName': 'Seafood Restaurants',
         'primary': True,
         'shortName': 'Seafood'}],
       'id': '563d44fccd1044ad67a744fb',
       'location': {'address': '5313 Yonge St',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 274,
        'formattedAddress': ['5313 Yonge St', 'Toronto ON M2N 5R4', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.773255217045026,
       

<h2>Create the Dataframe showing the different venues from the JSON file</h2>

In [118]:
# Get and define the categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# Define and create the Dataframe

venues = hv_json['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # normalise json

# filter columns
fc = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, fc]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Captain's Boil,Seafood Restaurant,43.773255,-79.413805
1,Aroma Espresso Bar,Café,43.769449,-79.413081
2,Loblaws,Grocery Store,43.768648,-79.412597
3,Dakgogi,Korean Restaurant,43.773010,-79.413875
4,Starbucks,Coffee Shop,43.768192,-79.413021


In [119]:
# Display amount of results returned by Foursquare

print('Foursquare returned {} venues.'.format(nearby_venues.shape[0]))

Foursquare returned 51 venues.


In [121]:
# Group by categories

hv_venues = nearby_venues.groupby('categories').count()

hv_venues

,name,lat,lng
categories,,,
American Restaurant,1,1,1
Arts & Crafts Store,1,1,1
Bank,1,1,1
Bubble Tea Shop,2,2,2
Burrito Place,1,1,1
Café,2,2,2
Coffee Shop,5,5,5
Dessert Shop,1,1,1
Discount Store,1,1,1


<h1>Reuse the venues in the whole of Toronto to peform OneHot encoding, create dataframes and do the K-Means clustering</h1>

In [122]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_tvenues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_tvenues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_tvenues)

In [125]:
wh_venues = getNearbyVenues(names=df_mer['Neighborhood'],
                                   latitudes=df_mer['Latitude'],
                                   longitudes=df_mer['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [127]:
#check the shape of the Dataframe

print(wh_venues.shape)
wh_venues.head()

(2248, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


<h2>Count the amount of venues for each area</h2>

In [134]:
wh_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,56,56,56,56,56,56


In [135]:
# Display the number of unique values
print('There are {} uniques categories.'.format(len(wh_venues['Venue Category'].unique())))

There are 277 uniques categories.


<h1>Perform the onehot encoding process</h1>

In [138]:
# one hot encoding
t_onehot = pd.get_dummies(wh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
t_onehot['Neighbourhood'] = wh_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [t_onehot.columns[-1]] + list(t_onehot.columns[:-1])
t_onehot = t_onehot[fixed_columns]

t_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malve

<h2>Group and shape the data</h2>

In [139]:
t_grouped = t_onehot.groupby('Neighbourhood').mean().reset_index()
t_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, Ki

In [141]:
t_grouped.shape

(101, 278)

# Find the top 10 venues for each neighborhood

In [143]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nvs = pd.DataFrame(columns=columns)
nvs['Neighbourhood'] = t_grouped['Neighbourhood']

for ind in np.arange(t_grouped.shape[0]):
    nvs.iloc[ind, 1:] = return_most_common_venues(t_grouped.iloc[ind, :], num_top_venues)

nvs.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,American Restaurant,Burger Joint,Cosmetics Shop,Hotel,Bakery
1,Agincourt,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Coffee Shop,Discount Store,Sandwich Place,Japanese Restaurant,Beer Store,Fried Chicken Joint,Pizza Place,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Pub,Pool,Dog Run,Dessert Shop


# Perfrom the K-Mean clustering

In [144]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set clusters
kclusters = 5

tgc = t_grouped.drop('Neighbourhood', 1)

# run clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tgc)

# check cluster labels
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [150]:
# Merge the dataframes


t_merged = df_mer

# merge 
t_merged = t_merged.join(nvs.set_index('Neighbourhood'), on='Neighborhood')

t_merged.head()

,PostalCode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497,0.0,Moving Target,History Museum,Bar,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711,0.0,Intersection,Spa,Pizza Place,Electronics Store,Medical Center,Breakfast Spot,Rental Car Location,Mexican Restaurant,Donut Shop,Diner
3,M1G,Woburn,Scarborough,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476,0.0,Bakery,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Bank,Thai Restaurant,Fried Chicken Joint,Yoga Studio,Drugstore,Doner Restaurant


In [152]:
t_merged[t_merged['Cluster Labels'].isnull()]

,PostalCode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,Upper Rouge,Scarborough,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,M9A,Islington Avenue,Etobicoke,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Plot on map

In [155]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

tmn = t_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tmn['Latitude'], tmn['Longitude'], tmn['Neighborhood'], tmn['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine 3 of the clusters

### Cluster 1

In [156]:
tmn.loc[tmn['Cluster Labels'] == 0, tmn.columns[[1] + list(range(5, tmn.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",0.0,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,"Highland Creek, Rouge Hill, Port Union",0.0,Moving Target,History Museum,Bar,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,"Guildwood, Morningside, West Hill",0.0,Intersection,Spa,Pizza Place,Electronics Store,Medical Center,Breakfast Spot,Rental Car Location,Mexican Restaurant,Donut Shop,Diner
3,Woburn,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,Cedarbrae,0.0,Bakery,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Bank,Thai Restaurant,Fried Chicken Joint,Yoga Studio,Drugstore,Doner Restaurant
6,"East Birchmount Park, Ionview, Kennedy Park",0.0,Discount Store,Hobby Shop,Department Store,Bus Station,Chinese Restaurant,Coffee Shop,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
7,"Clairlea, Golden Mile, Oakridge",0.0,Bus Line,Bakery,Park,Soccer Field,Bus Station,Intersection,Fast Food Restaurant,Doner Restaurant,Farmers Market,Falafel Restaurant
8,"Cliffcrest, Cliffside, Scarborough Village West",0.0,Motel,American Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
9,"Birch Cliff, Cliffside West",0.0,College Stadium,General Entertainment,Skating Rink,Café,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
10,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,Indian Restaurant,Pet Store,Furniture / Home Store,Vietnamese Restaurant,Light Rail Station,Latin American Restaurant,Chinese Restaurant,Thrift / Vintage Store,Donut Shop,Diner


### Cluster 2

In [157]:
tmn.loc[tmn['Cluster Labels'] == 1, tmn.columns[[1] + list(range(5, tmn.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,1.0,Playground,Construction & Landscaping,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
14,"Agincourt North, L'Amoreaux East, Milliken, St...",1.0,Park,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
48,"Moore Park, Summerhill East",1.0,Park,Trail,Tennis Court,Playground,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
50,Rosedale,1.0,Park,Playground,Trail,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
73,Humewood-Cedarvale,1.0,Trail,Park,Field,Hockey Arena,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


### Cluster 3

In [159]:
tmn.loc[tmn['Cluster Labels'] == 2, tmn.columns[[1] + list(range(5, tmn.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,"Humber Bay, King's Mill Park, Kingsway Park So...",2.0,Baseball Field,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Diner
97,"Emery, Humberlea",2.0,Baseball Field,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Diner
